### Getting info from Google News (NYTimes)


In [1]:
import http.client

conn = http.client.HTTPSConnection("google-news.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "83669c4ebamsh964bc077fea4c66p10dbedjsncb5951952abc",
    'x-rapidapi-host': "google-news.p.rapidapi.com"
    }

conn.request("GET", "/v1/source_search?source=nytimes.com&lang=en&country=US&from=2021-03-11&to=2021-03-12", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))


{"feed":{"title":"\"allinurl:nytimes.com after:2021-03-11 before:2021-03-12\" - Google News","updated":"Thu, 18 Mar 2021 16:52:46 GMT","link":"https://news.google.com/search?q=allinurl:nytimes.com+after:2021-03-11+before:2021-03-12&ceid=US:en&hl=en-US&gl=US","language":"en-US","subtitle":"Google News","rights":"2021 Google Inc."},"articles":[{"id":"CAIiEOtkoN4GdYGXjDl5uHvYmfwqFwgEKg8IACoHCAowjuuKAzCWrzww5oEY","title":"More U.S. States Expand Vaccine Eligibility - The New York Times","link":"https://www.nytimes.com/2021/03/11/world/covid-vaccine-eligibility.html","published":"Thu, 11 Mar 2021 10:07:46 GMT","sub_articles":[],"source":{"href":"https://www.nytimes.com","title":"The New York Times"}},{"id":"CAIiEEvwRkxcaeHgwAppWF8Vsu4qFwgEKg8IACoHCAowjuuKAzCWrzww5oEY","title":"EU Approves Johnson & Johnson Vaccine - The New York Times","link":"https://www.nytimes.com/2021/03/11/world/eu-johnson-johnson-covid-vaccine.html","published":"Thu, 11 Mar 2021 13:22:52 GMT","sub_articles":[],"source

Since the full article is not included, the link is taken from the articles and will be opened in another cell

In [2]:
import json
import requests

jsonNews = json.loads(data.decode("utf-8"))
jsonNews = jsonNews.get('articles')
links = []
requestedLinks = []

for i in range(len(jsonNews)):
    links.append(jsonNews[i].get('link'))
    temp = requests.get(links[i])
    requestedLinks.append(temp.content)

In [3]:
requestedLinks[0]

b'<!DOCTYPE html>\n<html lang="en" class="story nytapp-vi-article"  xmlns:og="http://opengraphprotocol.org/schema/">\n  <head>\n    <title data-rh="true">More U.S. States Expand Vaccine Eligibility - The New York Times</title>\n    <meta data-rh="true" property="article:published_time" content="2021-03-11T10:07:46.000Z"/><meta data-rh="true" property="article:modified_time" content="2021-03-12T15:04:08.994Z"/><meta data-rh="true" http-equiv="Content-Language" content="en"/><meta data-rh="true" name="robots" content="noarchive"/><meta data-rh="true" name="articleid" content="100000007647795"/><meta data-rh="true" name="nyt_uri" content="nyt://article/480b10b6-bdd1-59d9-bcd4-2e52786128e8"/><meta data-rh="true" name="pubp_event_id" content="pubp://event/365eeb5051a549db8c916e5867cc20fd"/><meta data-rh="true" name="description" content="Alaska, Georgia, Minnesota, New York, Ohio and Texas are among those offering shots to more people amid an energized national inoculation campaign."/><meta

In [4]:
from bs4 import BeautifulSoup

soup = []
for i in range(len(links)):
    soup.append(BeautifulSoup(requestedLinks[i], 'html.parser'))



In [5]:
jsonNews
jsonNewsExtracted = []
for i in range(len(jsonNews)):
    date = jsonNews[i].get('published')
    title = jsonNews[i].get('title')
    allAuthors = ""
    authors = soup[i].find_all("span", itemprop="name")
    for i in range (len(authors)):
        if i != len(authors)-1:
            allAuthors = allAuthors + authors[i].text + ', '
        else:
            allAuthors = allAuthors + authors[i].text
    try:
        full_article = soup[i].find(id = 'story').find('section').text
    except:
        pass
    jsonNewsExtracted.append({
            "date": date,
            "title": title,
            "full_article": full_article,
            "authors": allAuthors
        })

In [6]:
json_dump = json.dumps(jsonNewsExtracted, indent = 4) 

with open("news.json", "w") as outfile:
    outfile.write(json_dump)

In [7]:
from psaw import PushshiftAPI
import pandas as pd

api = PushshiftAPI()

ModuleNotFoundError: No module named 'psaw'

### Get the 5 most recent posts in all of Reddit

In [ ]:
posts = api.search_submissions(limit=5, filter=['full_link','author', 'title', 'subreddit', 'created_utc'])
results = list(posts)

In [ ]:
results[0]

### Get the most recent post from r/philippines

In [ ]:
posts = api.search_submissions(limit=5, subreddit="philippines", filter=['full_link','author', 'title', 'subreddit', 'created_utc'])
posts_df = pd.DataFrame([thing.d_ for thing in posts])

In [ ]:
posts_df

In [ ]:
posts_df.loc[0, 'full_link']

### Get posts from March 11 in r/philippines

In [ ]:
import datetime as dt

sub="philippines"
start="2021-03-11"

start_date=pd.to_datetime(start)

start_epoch=int(start_date.timestamp())

posts = api.search_submissions(limit=10, 
                               subreddit=sub, 
                               before=start_epoch,
                               filter=['full_link','author', 'title', 'subreddit', 'created_utc'])
posts_df = pd.DataFrame([thing.d_ for thing in posts])

In [ ]:
posts_df